In [146]:
# %load Find_SGLs.eQTL_Catalogue_Format.py
#!/usr/bin/env python

# %load ./scripts/loop_analysis/Annotate_Coloc_SNPs_with_Nearby_Genes_Loops.py
#!/usr/bin/env python

# # Make the master table
import os 
import sys
import pybedtools as pbt
import pandas as pd
import numpy as np
import subprocess as sp
import json
pbt.helpers.set_tempdir('/mnt/BioHome/jreyna/tmp/')
pbt.set_bedtools_path('/mnt/BioApps/bedtools/bin/')
bgzip = '/mnt/BioApps/tabix/tabix-0.2.6/bgzip'
tabix = '/mnt/BioApps/tabix/tabix-0.2.6/tabix'

In [147]:
os.chdir('/mnt/BioAdHoc/Groups/vd-ay/jreyna/projects/dchallenge/')

In [188]:
bedpe_6cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB']
bedpe_10cols = ['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB', 'name', 'score', 'strand1', 'strand2']

# ## default values for the command line
sys.argv = [0] * 100
# sys.argv[1] =  'results/main/sgls/T1D_34012112_Gaulton/Schmiedel_2018/monocyte_naive/eqtls.coloc_filtered.tsv.gz'
# sys.argv[2] = 'results/main/coloc/Results/Colocalization_SMKN/T1D_34012112_Gaulton/Schmiedel_2018/monocyte_naive/'
# sys.argv[2] += 'FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
# sys.argv[3] = 'results/main/h3k27ac_hichip/monocyte_naive/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.bed'
# sys.argv[4] = 'monocyte_naive'
# sys.argv[5] = 'results/refs/hg38/hg38.chrom.sizes'
# sys.argv[6] = 'results/refs/gencode/v30/gencode.v30.annotation.bed'
# sys.argv[7] = 'results/main/sgls/T1D_34012112_Gaulton/Schmiedel_2018/monocyte_naive/monocyte_naive'

sys.argv[1] = 'results/main/sgls/T1D_34012112_Gaulton/Quach_2016/monocyte_IAV/eqtls.coloc_filtered.tsv.gz'
sys.argv[2] = 'results/main/coloc/Results/Colocalization_SMKN/T1D_34012112_Gaulton/Quach_2016/monocyte_IAV/FINAL_Summary_Coloc_Gene_SNP_Pairs.bed'
sys.argv[3] = 'results/main/h3k27ac_hichip/monocyte_naive/FitHiChIP_S/FitHiChIP.interactions_FitHiC_Q0.01.grch38.bed'
sys.argv[4] = 'monocyte_IAV'
sys.argv[5] = 'results/refs/hg38/hg38.chrom.sizes'
sys.argv[6] = 'results/refs/gencode/v30/gencode.v30.annotation.bed'
sys.argv[7] = 'results/main/sgls/T1D_34012112_Gaulton/Quach_2016/monocyte_IAV/monocyte_naive'


#########################################################################################
# parsing the commandline arguments
#########################################################################################

eqtl_fn = sys.argv[1]
coloc_fn = sys.argv[2]
loop_fn = sys.argv[3]
celltype = sys.argv[4]
gs_fn = sys.argv[5]
genes_fn = sys.argv[6]
outdir = sys.argv[7]

In [189]:
#########################################################################################
# Load the colocalization data
#########################################################################################

# load the colocalization data
coloc = pd.read_table(coloc_fn)
coloc['sid'] = coloc['chr'].str.replace('chr', '')  + ':' + coloc['pos'].astype(str)
#coloc.rename(columns={'rs_id'})

In [190]:
coloc.head()

,chr,pos,pp_H0_Coloc_Summary,pp_H1_Coloc_Summary,pp_H2_Coloc_Summary,pp_H3_Coloc_Summary,pp_H4_Coloc_Summary,rs_id,variant_id,geneName,...,alt,AC,AF,AN,slope_se_snp,slope_gwas,slope_se_gwas,pval_nominal,SampleSize,sid
0,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000115263,...,G,2970,0.593051,5008,935884.206798,-0.118349,0.014721,9.030000e-16,520580,2:162368011
1,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000197635,...,G,2970,0.593051,5008,0.388094,-0.118349,0.014721,9.030000e-16,520580,2:162368011
2,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000115271,...,G,2970,0.593051,5008,3.428822,-0.118349,0.014721,9.030000e-16,520580,2:162368011
3,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000236841,...,G,2970,0.593051,5008,5594.125846,-0.118349,0.014721,9.030000e-16,520580,2:162368011
4,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000184611,...,G,2970,0.593051,5008,0.460995,-0.118349,0.014721,9.030000e-16,520580,2:162368011


In [191]:
coloc.shape

(32, 25)

In [192]:
# extract the most significant according the H4 
coloc_sgpairs = coloc[coloc['pp_H4_Coloc_Summary'] > 0.75]
coloc_sgpairs = coloc_sgpairs.loc[~coloc_sgpairs.duplicated(subset=['sid', 'geneName']),]

In [193]:
coloc_sgpairs.head()

,chr,pos,pp_H0_Coloc_Summary,pp_H1_Coloc_Summary,pp_H2_Coloc_Summary,pp_H3_Coloc_Summary,pp_H4_Coloc_Summary,rs_id,variant_id,geneName,...,alt,AC,AF,AN,slope_se_snp,slope_gwas,slope_se_gwas,pval_nominal,SampleSize,sid
0,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000115263,...,G,2970,0.593051,5008,935884.206798,-0.118349,0.014721,9.030000e-16,520580,2:162368011
1,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000197635,...,G,2970,0.593051,5008,0.388094,-0.118349,0.014721,9.030000e-16,520580,2:162368011
2,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000115271,...,G,2970,0.593051,5008,3.428822,-0.118349,0.014721,9.030000e-16,520580,2:162368011
3,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000236841,...,G,2970,0.593051,5008,5594.125846,-0.118349,0.014721,9.030000e-16,520580,2:162368011
4,chr2,162368011,6.793362e-75,1.005820e-73,0.008265,0.121498,0.870237,rs984971,2:162368011:A:G,ENSG00000184611,...,G,2970,0.593051,5008,0.460995,-0.118349,0.014721,9.030000e-16,520580,2:162368011


In [194]:
coloc_sig_full = coloc_sgpairs.copy(deep=True)
coloc_sgpairs.rename(columns={'pos': 'end'}, inplace=True)
coloc_sgpairs.loc[:, 'start'] = coloc_sgpairs.loc[:, 'end'] - 1

print('There are {} colocalized SNP-gene pairs'.format(coloc_sgpairs.shape[0]))
print('There are {} colocalized SNPs'.format(coloc_sgpairs.sid.nunique()))
print('There are {} colocalized genes'.format(coloc_sgpairs.geneName.nunique()))

There are 22 colocalized SNP-gene pairs
There are 2 colocalized SNPs
There are 22 colocalized genes


In [195]:
coloc_snps = coloc_sgpairs[['chr', 'start', 'end', 'rs_id', 'variant_id', 'sid']]

In [196]:
coloc_snps = coloc_snps.loc[~coloc_snps.duplicated(subset='sid'), :]

In [197]:
coloc_snps_pbt = pbt.BedTool.from_dataframe(coloc_snps.iloc[:, [0,1,2,5]]).sort()

In [198]:
#########################################################################################
# ## Load the gene data
#########################################################################################
print('# Load the gene data')

# load the gencode coords
cols = ['chrom', 'start', 'end', 'strand', 'type', 'gene_id', 'gname']
gencode = pd.read_table(genes_fn, header=None, names=cols)

# extract just the genes
genes_df = gencode.loc[gencode.type.isin(['gene'])]
genes_df = genes_df.loc[~genes_df.duplicated(subset='gene_id'), :]
genes_df.loc[:, 'chrom'] = genes_df['chrom'].astype(str)
genes_df = genes_df.iloc[:, [0,1,2,6,5,3]]

# create a copy of the original gene bed before coordinate shrinking
orig_genes_df = genes_df.copy()

# convert the start/end position into start/end for the TSS
# if the gene is + then the start is uses as the tss otherwise
# the end is used as the tss
genes_df.loc[(genes_df.strand == '+'), 'end'] = genes_df.loc[(genes_df.strand == '+'), 'start']
genes_df.loc[(genes_df.strand == '+'), 'start'] = genes_df.loc[(genes_df.strand == '+'), 'start'] - 1
genes_df.loc[(genes_df.strand == '-'), 'end'] = genes_df.loc[(genes_df.strand == '-'), 'end']
genes_df.loc[(genes_df.strand == '-'), 'start'] = genes_df.loc[(genes_df.strand == '-'), 'end'] - 1

# make a genes pbt for intersection
print("# make a genes pbt for intersection")
print(genes_df.head())
genes_pbt = pbt.BedTool.from_dataframe(genes_df).sort()

print('There are {} genes in this GTF-derived file.'.format(genes_df.shape[0]))

# Load the gene data
# make a genes pbt for intersection
   chrom  start    end        gname          gene_id strand
0   chr1  11868  11869      DDX11L1  ENSG00000223972      +
12  chr1  29569  29570       WASH7P  ENSG00000227232      -
25  chr1  17435  17436    MIR6859-1  ENSG00000278267      -
28  chr1  29553  29554  MIR1302-2HG  ENSG00000243485      +
36  chr1  30365  30366    MIR1302-2  ENSG00000284332      +
There are 58825 genes in this GTF-derived file.


In [199]:
#########################################################################################
# Find all genes +/- 500kb of a colocalized SNP
#########################################################################################
print("# Find all genes +/- 500kb")

# get a list of gene names within +- 500kb of the SNPs
fivekb_genes = coloc_snps_pbt.slop(b=500000, g=gs_fn)

# Find all genes +/- 500kb


In [200]:
fivekb_genes.to_dataframe()

,chrom,start,end,name
0,chr2,161868010,162868011,2:162368011
1,chr9,3796429,4796430,9:4296430


In [201]:
fivekb_genes = fivekb_genes.intersect(genes_pbt, wa=True, wb=True)
fivekb_genes = fivekb_genes.to_dataframe()
fivekb_genes = fivekb_genes.iloc[:, [0,1,2,4,5,6,3,7,8,9]]

fivekb_genes.columns = bedpe_6cols + ['rs_id', 'gname', 'gid', 'strand2']
fivekb_genes['strand1'] = '+'
fivekb_genes['name'] = fivekb_genes['rs_id'] + '_' + fivekb_genes['gname']
fivekb_genes['score'] = '.'
new_order = bedpe_10cols + ['rs_id', 'gname', 'gid']
fivekb_genes = fivekb_genes[new_order]

fivekb_genes['startA'] += 500000
fivekb_genes['endA'] -= 500000
fivekb_genes['sid'] = fivekb_genes['chrA'].str.replace('chr', '') + ':' + fivekb_genes['endA'].astype(str)

print('There are {} colocalized snp-gene pairs within +/- 500kb.'.format(fivekb_genes.shape[0]))

There are 30 colocalized snp-gene pairs within +/- 500kb.


In [202]:
#########################################################################################
# Find the closest gene
#########################################################################################
print('# Find the closest gene')

closest_gene = coloc_sig_pbt.closest(genes_pbt, d=True)
closest_gene = closest_gene.to_dataframe().iloc[:, [0,1,2,4,5,6,3,7,8,9]]
closest_gene.columns = bedpe_6cols + ['rs_id', 'gname', 'gid', 'dist']
closest_gene['sid'] = closest_gene['chrA'].str.replace('chr', '') + ':' + closest_gene['endA'].astype(str)
closest_gene.set_index(['sid', 'gname'], inplace=True)

# Find the closest gene


In [203]:
tmp_loops

,chr1,s1,e1,chr2,s2,e2,name,score,strand1,strand2
0,chr1,999620,1004620,chr1,1019620,1024620,.,4.851310e-06,.,.
1,chr1,1778561,1783561,chr1,3793436,3798436,.,4.646481e-08,.,.
2,chr1,26278509,26283509,chr1,26318509,26323509,.,3.598136e-06,.,.
3,chr10,44934552,44939552,chr10,44954552,44959552,.,5.488571e-06,.,.
4,chr10,44944552,44949552,chr10,44959552,44964552,.,5.173212e-06,.,.
...,...,...,...,...,...,...,...,...,...,...
84137,chr10,43434552,43439552,chr10,45379552,45384552,.,4.792100e-08,.,.
84138,chr10,43434552,43439552,chr10,45714552,45719552,.,4.075525e-08,.,.
84139,chr10,43434552,43439552,chr10,44954552,44959552,.,4.851310e-06,.,.
84140,chr10,44934552,44939552,chr10,44959552,44964552,.,4.528059e-06,.,.


In [205]:
#########################################################################################
# Load the H3K27ac HiChIP loops
#########################################################################################
print('# Get the loops')

# load the loop data
loops = pd.read_table(loop_fn)

# Get the loops


In [206]:
loops

,chr1,s1,e1,chr2,s2,e2,cc,Coverage1,isPeak1,Bias1,...,Bias2,Mapp2,GCContent2,RESites2,p,exp_cc_Bias,p_Bias,dbinom_Bias,P-Value_Bias,Q-Value_Bias
0,chr1,999620,1004620,chr1,1019620,1024620,31,590,1,0.442683,...,0.354897,0,0,0,4.851310e-06,9.241261,8.645438e-08,1.021450e-08,1.429445e-08,3.577668e-06
1,chr1,1778561,1783561,chr1,3793436,3798436,10,10203,1,7.655420,...,0.282117,0,0,0,4.646481e-08,1.985142,1.857152e-08,3.597630e-05,4.373368e-05,5.051998e-03
2,chr1,26278509,26283509,chr1,26318509,26323509,517,12458,1,9.347371,...,3.311872,0,0,0,3.598136e-06,392.886045,3.675550e-06,3.215832e-10,1.307941e-09,3.909070e-07
3,chr10,44934552,44939552,chr10,44954552,44959552,124,658,1,0.493704,...,1.306291,0,0,0,5.488571e-06,31.570973,2.953546e-07,1.053721e-35,1.408527e-35,4.933339e-32
4,chr10,44944552,44949552,chr10,44959552,44964552,73,658,1,0.493704,...,0.741307,0,0,0,5.173212e-06,19.034114,1.780690e-07,3.084248e-21,4.145798e-21,4.803915e-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84137,chr10,43434552,43439552,chr10,45379552,45384552,12,8964,1,6.725785,...,2.393491,0,0,0,4.792100e-08,2.755553,2.577891e-08,2.543585e-05,3.212350e-05,3.852477e-03
84138,chr10,43434552,43439552,chr10,45714552,45719552,11,8964,1,6.725785,...,0.951779,0,0,0,4.075525e-08,2.239503,2.095113e-08,1.896306e-05,2.322613e-05,2.893782e-03
84139,chr10,43434552,43439552,chr10,44954552,44959552,96,476,1,0.357148,...,1.306291,0,0,0,4.851310e-06,21.079813,1.972071e-07,8.709230e-33,1.111877e-32,3.184313e-29
84140,chr10,44934552,44939552,chr10,44959552,44964552,36,476,1,0.357148,...,0.741307,0,0,0,4.528059e-06,12.629390,1.181512e-07,3.929659e-08,5.926719e-08,1.321116e-05


In [207]:
tmp_loops = loops[['chr1', 's1', 'e1', 'chr2', 's2', 'e2']]
tmp_loops.rename(columns={'p': 'score'}, inplace=True)
tmp_loops.loc[:, 'name'] = '.'
tmp_loops.loc[:, 'score'] = loops['p']
tmp_loops.loc[:, 'strand1'] = '.'
tmp_loops.loc[:, 'strand2'] = '.'
loops = pbt.BedTool.from_dataframe(tmp_loops)
print(tmp_loops.shape)

/mnt/BioHome/jreyna/tmp//ipykernel_61203/4187765372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_loops.rename(columns={'p': 'score'}, inplace=True)
/mnt/BioHome/jreyna/tmp//ipykernel_61203/4187765372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_loops.loc[:, 'name'] = '.'
/mnt/BioHome/jreyna/tmp//ipykernel_61203/4187765372.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

(84142, 10)


In [208]:
coloc_snps

,chr,start,end,rs_id,variant_id,sid
0,chr2,162368010,162368011,rs984971,2:162368011:A:G,2:162368011
20,chr9,4296429,4296430,rs10814917,9:4296430:A:G,9:4296430


In [211]:
#########################################################################################
# Find snp-gene pairs overlapping loops
#########################################################################################

# re-arranging to fit bedpe format
fivekb_gloops = fivekb_genes.copy()

# loading into pbt
fivekb_gloops = pbt.BedTool.from_dataframe(fivekb_gloops)
fivekb_gloops = fivekb_gloops.pair_to_pair(loops, type='both', slop=20000, **{'is':True})
#fivekb_gloops = fivekb_gloops.pair_to_pair(loops, type='both', slop=7500, **{'is':True})

fivekb_gloops = fivekb_gloops.to_dataframe(disable_auto_names=True, header=None)

if len(fivekb_gloops) > 0:
    fivekb_gloops_set = fivekb_gloops.iloc[:, [13,12]]
    fivekb_gloops_uniq = set([tuple(x) for x in fivekb_gloops_set.values.tolist()])
else:
    print('WARNING: Found no overlap between the genes and loops.')
    fivekb_gloops_uniq = set()

print('There are {} SNP-Gene pairs with a loop.'.format(len(fivekb_gloops_uniq)))

There are 2 SNP-Gene pairs with a loop.


In [10]:
#########################################################################################
# Construct master table
#########################################################################################
print('# Construct master table')

# begin making the master
master = fivekb_genes.copy()
master['sid'] = master['chrA'].str.replace('chr', '') + ':' + master['endA'].astype(str)

print('Master is starting with {} snp-gene pairs.'.format(master.shape[0]))

# Construct master table
Master is starting with 942 snp-gene pairs.


In [11]:
#coloc_sig_df['sid'] = coloc_sig_df['chr'].str.replace('chr', '') + ':'

In [12]:
# #### Add eqtl results 
print("# #### Add eqtl results")

# load the eQTL's
eqtls = pd.read_table(eqtl_fn, header=0).head(10000)
# old eqtls.columns = ['eqtl_gname', 'nvar', 'shape1', 'shape2', 'dummy',
#                 'sid', 'dist', 'npval', 'slope', 'ppval', 'bpval', 'qval']
# new eqtls.columns
# molecular_trait_id, chromosome, position, ref, alt, variant, ma_samples,
# maf, pvalue, beta, se, type, ac, an, r2, molecular_trait_object_id, gene_id, median_tpm, rsid, dist, fdr
eqtls['sid'] = eqtls['chromosome'].astype(str) + ':' + eqtls['position'].astype(str)

print('There are {} eQTLs.'.format(eqtls.shape[0]))

# #### Add eqtl results
There are 10000 eQTLs.


In [13]:
# need to use outer or else you exclude some eQTL's
master = master.merge(eqtls, left_on=['sid', 'gname'], right_on=['sid', 'molecular_trait_id'], how='outer')

In [14]:
# add column to filter on eqtl snp status
master['is_eqtl_pair'] = (~master['pvalue'].isna()).astype(int)

In [15]:
# add gene names to entries with a missing name (after adding eQTL info)
master.loc[master.gname.isna(), 'gname'] = master.loc[master.gname.isna(), 'molecular_trait_id']

In [16]:
master

,chrA,startA,endA,chrB,startB,endB,name,score,strand1,strand2,...,ac,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,dist,fdr,is_eqtl_pair
0,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,396.0,0.89079,ENSG00000162542,ENSG00000162542,2.315,rs142679596,688135.0,0.007247,1
10938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,104.0,396.0,0.99999,ENSG00000127481,ENSG00000127481,69.666,rs6693149,90173.0,0.000315,1
10939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96.0,396.0,0.99759,ENSG00000127481,ENSG00000127481,69.666,rs6696220,86426.0,0.002288,1
10940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,102.0,396.0,0.99582,ENSG00000127481,ENSG00000127481,69.666,rs6688770,86155.0,0.000315,1


In [17]:
# add missing chrA, chrB, startA and endA data for the eQTL rows
master.loc[master.chrA.isna(), 'chrA'] = 'chr' + master.loc[master.chrA.isna(), 'sid'].str.replace(':[0-9]+', '')
master.loc[master.chrB.isna(), 'chrB'] = 'chr' + master.loc[master.chrB.isna(), 'sid'].str.replace(':[0-9]+', '')
master.loc[master.startA.isna(), 'startA'] = (master.loc[master.startA.isna(), 'sid'].str.replace('[0-9]+:', '')).astype(int)
master.loc[master.startA.isna(), 'startA'] -= 1     
master.loc[master.endA.isna(), 'endA'] = master.loc[master.endA.isna(), 'sid'].str.replace('[0-9]+:', '')

print('After outer merging with eqtls, master has {} rows.'.format(master.shape[0]))

After outer merging with eqtls, master has 10942 rows.


/mnt/BioHome/jreyna/tmp//ipykernel_61203/3726262882.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  master.loc[master.chrA.isna(), 'chrA'] = 'chr' + master.loc[master.chrA.isna(), 'sid'].str.replace(':[0-9]+', '')
/mnt/BioHome/jreyna/tmp//ipykernel_61203/3726262882.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  master.loc[master.chrB.isna(), 'chrB'] = 'chr' + master.loc[master.chrB.isna(), 'sid'].str.replace(':[0-9]+', '')
/mnt/BioHome/jreyna/tmp//ipykernel_61203/3726262882.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  master.loc[master.startA.isna(), 'startA'] = (master.loc[master.startA.isna(), 'sid'].str.replace('[0-9]+:', '')).astype(int)
/mnt/BioHome/jreyna/tmp//ipykernel_61203/3726262882.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  master.loc[master.endA.isna(),

In [18]:
master

,chrA,startA,endA,chrB,startB,endB,name,score,strand1,strand2,...,ac,an,r2,molecular_trait_object_id,gene_id,median_tpm,rsid,dist,fdr,is_eqtl_pair
0,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,chr18,12780463.0,12780464.0,chr18,12288307.0,12288308.0,rs9952991_AP005264.1,.,+,+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10937,chr1,19111810.0,19111810,chr1,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,396.0,0.89079,ENSG00000162542,ENSG00000162542,2.315,rs142679596,688135.0,0.007247,1
10938,chr1,19120093.0,19120093,chr1,NaN,NaN,NaN,NaN,NaN,NaN,...,104.0,396.0,0.99999,ENSG00000127481,ENSG00000127481,69.666,rs6693149,90173.0,0.000315,1
10939,chr1,19123840.0,19123840,chr1,NaN,NaN,NaN,NaN,NaN,NaN,...,96.0,396.0,0.99759,ENSG00000127481,ENSG00000127481,69.666,rs6696220,86426.0,0.002288,1
10940,chr1,19124111.0,19124111,chr1,NaN,NaN,NaN,NaN,NaN,NaN,...,102.0,396.0,0.99582,ENSG00000127481,ENSG00000127481,69.666,rs6688770,86155.0,0.000315,1


In [19]:
master.loc[~master.molecular_trait_id.isna(), 'gid'] = master.loc[~master.molecular_trait_id.isna(), 'molecular_trait_id']

In [20]:
# add back the original gene start and end
master = master.merge(orig_genes_df[['start', 'end', 'gene_id', 'strand']], left_on='gid', right_on='gene_id')

# convert the startB/endB position into startB/endB for the TSS
# if the gene is + then the startB is uses as the tss otherwise
# the endB is used as the tss
master.loc[(master.strand == '+'), 'endB'] = master.loc[(master.strand == '+'), 'start']
master.loc[(master.strand == '+'), 'startB'] = master.loc[(master.strand == '+'), 'start'] - 1

master.loc[(master.strand == '-'), 'endB'] = master.loc[(master.strand == '-'), 'end']
master.loc[(master.strand == '-'), 'startB'] = master.loc[(master.strand == '-'), 'end'] - 1

# convert the coordinates from floats to ints
master.startA = master.startA.astype(int)
master.startB = master.startB.astype(int)
master.endA = master.endA.astype(int)
master.endB = master.endB.astype(int)

master.rename(columns={'start': 'gene_start', 'end': 'gene_end', 'strand': 'gene_strand'}, inplace=True)

# #### Add info about closests gene
print("# #### Add info about closests gene")

# check for the closets gene
closets_check = [0] * master.shape[0]
for i, sr in master.iterrows():

    # check closest gene
    rs_gene = (sr.sid, sr.gname)
    if rs_gene in closest_gene.index:
        closets_check[i] = 1

master['is_closest_gene'] = closets_check

# #### Add info about closests gene


In [21]:
# #### Add colocalization data
print("# #### Add colocalization data")

# add colocalization data for SNP and is_coloc_snp columns
tmp_coloc = coloc_sig_full[['chr', 'pos', # NEW
 'pp_H0_Coloc_Summary',
 'pp_H1_Coloc_Summary',
 'pp_H2_Coloc_Summary',
 'pp_H3_Coloc_Summary',
 'pp_H4_Coloc_Summary',
 'rs_id',
 'geneName',
 'ref',
 'alt',
 'AC',
 'AF',
 'AN',
 'slope_gwas',
 'slope_se_gwas',
 'pval_nominal',
 'SampleSize']]
tmp_coloc.rename(columns={'slope_gwas': 'gwas_slope',
                          'slope_se_gwas': 'gwas_slope_se',
                          'pval_nominal': 'gwas_pval_nominal',
                          'geneName': 'gname'}, inplace=True)

tmp_coloc['sid'] = tmp_coloc['chr'].str.replace('chr', '') + ':' +  tmp_coloc['pos'].astype(str) # NEW

# #### Add colocalization data


/mnt/BioHome/jreyna/tmp//ipykernel_61203/2650951363.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_coloc.rename(columns={'slope_gwas': 'gwas_slope',
/mnt/BioHome/jreyna/tmp//ipykernel_61203/2650951363.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_coloc['sid'] = tmp_coloc['chr'].str.replace('chr', '') + ':' +  tmp_coloc['pos'].astype(str) # NEW


In [22]:
master = master.merge(tmp_coloc, left_on=['sid', 'gid'], right_on=['sid', 'gname'], how='left')

In [23]:
# add column to filter on coloc snp status
master['is_coloc_pair'] = (~master['pp_H4_Coloc_Summary'].isna()).astype(int)

print('After left merging master with the colocalization table there are {} entries.'.format(master.shape[0]))

After left merging master with the colocalization table there are 10942 entries.


In [24]:
# #### Add loop data
print('# #### Add loop data')

# check for the loop gene
loop_check = [0] * master.shape[0]
for i, sr in master.iterrows():

    # check closest gene
    rs_gene = (sr.sid, sr.gid)
    if rs_gene in fivekb_gloops_uniq:
        loop_check[i] = 1       

master['has_fithichip_loop'] = loop_check
print('There are {} SNP-Gene loops.'.format(sum(loop_check)))

# #### Add loop data
There are 75 SNP-Gene loops.


In [25]:
# #### Do the final reordering and saving

# Include PC HiC data when it's available
final_cols = [
    'sid', 
    'rsid',
    'gid',
    'chrA',
    'endA',
    'startB',
    'endB',
    'is_eqtl_pair',
    'is_coloc_pair',
    'is_closest_gene',
    'has_fithichip_loop',
    'pvalue', 
    'beta', 
    'fdr',
    'dist',
    'pp_H0_Coloc_Summary',
    'pp_H1_Coloc_Summary',
    'pp_H2_Coloc_Summary',
    'pp_H3_Coloc_Summary',
    'pp_H4_Coloc_Summary',
    'gene_start',
    'gene_end',
    'ref_x', 
    'alt_x',
    'AC',
    'AF',
    'AN',
    'gwas_slope',
    'gwas_slope_se',
    'gwas_pval_nominal',
    'SampleSize']

In [26]:
master = master[final_cols]
master.rename(columns={'chrA':'chrom', 'endA': 'snp_pos', 'ref_x': 'ref', 'alt_x': 'alt',
                       'startB': 'tss_start', 'endB': 'tss_end',
                       'gname': 'gene_name', 'gid': 'gene_id'}, inplace=True)

master.sort_values(['chrom', 'snp_pos', 'tss_start', 'rs_id'], inplace=True)
master.snp_pos = master.snp_pos.astype(int)

# write out the master data
fn = os.path.join(outdir, 'master.tsv')
master.to_csv(fn, sep='\t', header=True, index=False)

fn = os.path.join(outdir, 'master.xlsx')
excel_master = master.sort_values('rs_id').set_index('rs_id')
excel_master.to_excel(fn, na_rep='nan')

KeyError: 'rs_id'

In [ ]:
#########################################################################################
# Make WashU files 
#########################################################################################

def bedpe_to_WashU_longrange(fn, df):
    """
        Convert from a loop bedpe file into WashU longrange, 
        includes bgzip and tabix of the fn. 
        
        Params
        -------
        fn: str
            path to the longrange output file (without gz)
            
        df: dataframe
            columns 1-6 are as expected and column 7 is the p or q-value. 
            
        Output
        ------
        gzfn: str
            path to the longrange with bgzip compression
        tabix_fn: str
            path to the index of the longrange file
            
    """

    # parsing the data into WashU longrage format
    data = []
    for sr in df.values.tolist():

        # calculate the -log(FDR)
        qval = -np.log(sr[6])

        # get the first pair data
        second_pair_str = '{}:{}-{},{:.5f}'.format(*sr[3:6], qval)
        first_row = sr[0:3] + [second_pair_str]

        # get the second pair data
        first_pair_str = '{}:{}-{},{:.5f}'.format(*sr[0:3], qval)
        second_row = sr[3:6] + [first_pair_str]

        # add each data row
        data.append(first_row)
        data.append(second_row)

    data = sorted(data, key=lambda x: (x[0], x[1], x[2]))

    # writing out the data
    with open(fn, 'w') as f:
        for line in data:
            info = [str(x) for x in line]
            info = '\t'.join(info)
            f.write(info + '\n')
            
    # run bgzip
    cmd = '{} {}'.format(bgzip, fn)
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())
    
    # run tabix
    lrange_gzfn = fn + '.gz'
    cmd = '{} -f {}'.format(tabix, lrange_gzfn)
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())

    print('Created the gzfn: {}'.format(fn + '.gz'))
    print('Created the tabix: {}'.format(fn + '.gz.tbi'))

def bed_WashU_bedgz(fn, df):
    """
        Convert from a bed dataframe into WashU longrange file 
        includes bgzip and tabix of the fn. 
        
        Params
        -------
        fn: str
            path to the longrange output file (without gz)
            
        df: dataframe
            columns 1-3 are as expected and column 7 is the p or q-value. 
            
        Output
        ------
        gzfn: str
            path to the longrange with bgzip compression
        tabix_fn: str
            path to the index of the longrange file
            
    """

    # parsing the data into WashU longrage format
    data = []
    for sr in df.values.tolist():
        data.append(sr[0:4])
    data = sorted(data, key=lambda x: (x[0], x[1], x[2]))

    # writing out the data
    with open(fn, 'w') as f:
        for line in data:
            info = [str(x) for x in line]
            info = '\t'.join(info)
            f.write(info + '\n')
            
    # run bgzip
    cmd = '{} {}'.format(bgzip, fn)
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())
    
    # run tabix
    gzfn = fn + '.gz'
    cmd = '{} -f {}'.format(tabix, gzfn)
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())

    print('Created the gzfn: {}'.format(fn + '.gz'))
    print('Created the tabix: {}'.format(fn + '.gz.tbi'))

def bed_to_WashU_refbed(fn, df):
    """ 
        Convert from a bed dataframe into WashU longrange file 
        includes bgzip and tabix of the fn. 
        
        Params
        -------
        fn: str
            path to the longrange output file (without gz)
            
        df: dataframe
            columns 1-3 are as expected and column 7 is the p or q-value. 
            
        Output
        ------
        gzfn: str
            path to the longrange with bgzip compression
        tabix_fn: str
            path to the index of the longrange file
            
    """

    # parsing the data into WashU longrage format
    data = df.values.tolist()
    data = sorted(data, key=lambda x: (x[0], x[1], x[2]))

    # writing out the data
    with open(fn, 'w') as f:
        for line in data:
            info = [str(x) for x in line]
            info = '\t'.join(info)
            f.write(info + '\n')
    
    # run bgzip
    cmd = '{} -f {}'.format(bgzip, fn) 
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())

    # run tabix
    gzfn = fn + '.gz'
    cmd = '{} {}'.format(tabix, gzfn)
    print(cmd)
    job = sp.Popen(cmd, stderr=sp.PIPE,stdout=sp.PIPE, shell=True)

    out, err = job.communicate()
    print('out:', out.decode())
    print('err:', err.decode())

    print('Created the gzfn: {}'.format(fn + '.gz'))
    print('Created the tabix: {}'.format(fn + '.gz.tbi'))


# make the refbed link for genes (status: running)
final_sg_cols = ['chrom', 'gene_start' ,'gene_end', 'gene_name', 'gene_strand']
final_sg_genes = master.loc[(master.has_fithichip_loop == 1), final_sg_cols]

final_sg_genes.gene_start = final_sg_genes.gene_start.astype(int)
final_sg_genes.gene_end = final_sg_genes.gene_end.astype(int)

final_sg_genes = final_sg_genes.loc[~final_sg_genes.duplicated()]
final_sg_genes['chr'] = final_sg_genes['chrom'] 
final_sg_genes['transcript_start'] = final_sg_genes['gene_start']
final_sg_genes['transcript_stop'] = final_sg_genes['gene_end']
final_sg_genes['translation_start'] = final_sg_genes['gene_start']
final_sg_genes['translation_stop'] = final_sg_genes['gene_end']
final_sg_genes['strand'] = final_sg_genes['gene_strand']
final_sg_genes['gene_name'] = final_sg_genes['gene_name']
final_sg_genes['transcript_id'] = final_sg_genes['gene_name']
final_sg_genes['type'] = 'coding'
final_sg_genes['exon_gene_start'] = final_sg_genes['gene_start']
final_sg_genes['exon_stops'] = final_sg_genes['gene_end']
refcols = ['chr', 'transcript_start', 'transcript_stop', 'translation_start',
           'translation_stop', 'strand', 'gene_name', 'transcript_id',
           'type', 'exon_gene_start', 'exon_stops']
final_sg_genes = final_sg_genes.loc[:, refcols]
sg_genes_fn = os.path.join(outdir, 'gs_genes.bed')
bed_to_WashU_refbed(sg_genes_fn, final_sg_genes)


# make the longrange link for raw fithichip data (status: running)
loop_gz = os.path.abspath(loop_fn.replace('.bed', '_WashU.bed.gz'))
loop_tbi = os.path.abspath(loop_fn.replace('.bed', '_WashU.bed.gz.tbi'))
loop_gz_link = os.path.join(outdir, os.path.basename(loop_gz))
loop_tbi_link = os.path.join(outdir, os.path.basename(loop_tbi))
if not os.path.exists(loop_gz_link):
    os.link(loop_gz, loop_gz_link)
    os.link(loop_tbi, loop_tbi_link)


# ## All 5kb Washu Files

# make the longrange link for snp-gene fivekb pairs (status: running)
#fivekb_lrange = fivekb_lrange[fivekb_lrange.rs_id.notna()].reset_index(drop=True)
fivekb_lrange = fivekb_genes.copy()

# convert full for viz
fivekb_lrange = fivekb_lrange[['chrA', 'startA', 'endA', 'chrB', 'startB', 'endB']]
fivekb_lrange.iloc[:, 1] -= 1 
fivekb_lrange['score'] = 0.01

fivekb_snp_gene_pairs_fn = os.path.join(outdir, '5kb.snp_gene_pairs.bed')
bedpe_to_WashU_longrange(fivekb_snp_gene_pairs_fn, fivekb_lrange)

# plot all snp-gene pairs with a loop
fivekb_gloops_lrange = fivekb_gloops.iloc[:, [11,12,13,14,15,16,18]]

fivekb_snp_gene_loops_fn = os.path.join(outdir, '5kb.snp_gene_loops.bed')
bedpe_to_WashU_longrange(fivekb_snp_gene_loops_fn, fivekb_gloops_lrange)


# ## eQTL WashU Files

# make the longrange link for snp-gene eQTL pairs (status: running)
eqtl_lrange = master.loc[master.is_eqtl_pair == 1]
eqtl_lrange = eqtl_lrange[eqtl_lrange.rs_id.notna()].reset_index(drop=True)

# convert full for viz
eqtl_snp_gene_pairs = eqtl_lrange[['chrom', 'snp_pos', 'snp_pos', 'chrom', 'tss_start', 'tss_end', 'gene_name']]
eqtl_snp_gene_pairs.iloc[:, 1] -= 1 
eqtl_snp_gene_pairs['score'] = 0.01

eqtl_snp_gene_pairs_fn = os.path.join(outdir, 'eqtl.snp_gene_pairs.bed')
bedpe_to_WashU_longrange(eqtl_snp_gene_pairs_fn, eqtl_snp_gene_pairs.iloc[:, [0,1,2,3,4,5,7]])

# make the bed for egenes only (status: running)
eqtl_genes = eqtl_lrange.loc[:, ['chrom', 'gene_start', 'gene_end', 'gene_name']]
eqtl_genes_only_fn = os.path.join(outdir, 'eqtl.genes_only.bed')
bed_WashU_bedgz(eqtl_genes_only_fn, eqtl_genes)


# In[49]:


# In[45]:


# make the bed for eSNPs only (status: running)
eqtl_snps = eqtl_lrange.loc[:, ['chrom', 'snp_pos', 'snp_pos', 'rs_id']]
eqtl_snps.iloc[:, 1] -= 1
eqtl_snps = pbt.BedTool.from_dataframe(eqtl_snps)
eqtl_snps = eqtl_snps.slop(b=500, g=gs_fn).to_dataframe()
eqtl_snps_only_fn = os.path.join(outdir, 'eqtl.snps_only.bed')
bed_WashU_bedgz(eqtl_snps_only_fn, eqtl_snps)


# In[50]:


# In[46]:


# # make the longrange link for snp-gene eQTL loops (status: didn't add loop coordinates to master.)
# eqtl_loops = master.loc[master.is_eqtl_pair == 1]


# ## Coloc WashU Files

# In[51]:


# In[47]:


# make the bed for colocalized SNPs (status: running)
final_snps = coloc_sig_df.copy()
final_snps = final_snps.loc[~final_snps.duplicated()]
final_snps = pbt.BedTool.from_dataframe(final_snps)
final_snps = final_snps.slop(b=500, g=gs_fn)
final_snps = final_snps.to_dataframe()[0:4]


# In[52]:


# In[48]:


coloc_snps_only_fn = os.path.join(outdir, 'coloc.snps_only.bed')
bed_WashU_bedgz(coloc_snps_only_fn, final_snps)


# In[53]:


# In[49]:


# make the bed for colocalized genes (status: running)
final_coloc_snp_genes_pairs = master[(master.is_coloc_pair == 1)]
final_coloc_genes = final_coloc_snp_genes_pairs[['chrom', 'gene_start' ,'gene_end', 'gene_name']]
coloc_genes_only_fn = os.path.join(outdir, 'coloc.genes_only.bed')
bed_WashU_bedgz(coloc_genes_only_fn, final_coloc_genes)


# In[54]:


# In[50]:


# make the longrange for colocalized snp-gene pairs (status: running)
final_coloc_snp_genes_pairs_out = final_coloc_snp_genes_pairs[['chrom', 'snp_pos', 'snp_pos', 'chrom', 
                                                               'tss_start', 'tss_end', 'gene_name']]
final_coloc_snp_genes_pairs_out.iloc[:, 1] -= 1
final_coloc_snp_genes_pairs_out.iloc[:, 6] = 0.01

coloc_snp_gene_pairs_fn = os.path.join(outdir, 'coloc.snp_gene_pairs.bed')
bedpe_to_WashU_longrange(coloc_snp_gene_pairs_fn, final_coloc_snp_genes_pairs_out)


# In[55]:


# In[51]:


# make the longrange for colocalized snp-gene loops (status: running)
final_coloc_snp_genes_loops = fivekb_gloops[(fivekb_gloops[10].isin(final_coloc_snp_genes_pairs['rs_id'])) & 
              (fivekb_gloops[12].isin(final_coloc_snp_genes_pairs['gene_id']))]
final_coloc_snp_genes_loops_out = final_coloc_snp_genes_loops.iloc[:, [14,15,16,17,18,19,21]]

coloc_snp_gene_loops_fn = os.path.join(outdir, 'coloc.snp_gene_loops.bed')
bedpe_to_WashU_longrange(coloc_snp_gene_loops_fn, final_coloc_snp_genes_loops_out)


# In[56]:


# In[52]:

final_coloc_snp_genes_loops_out


# In[57]:


# In[53]:


# make the loop anchors as bed files (status: developing)
left = final_coloc_snp_genes_loops[[14,15,16,6]].T.reset_index(drop=True).T
left[3] = 'L-' + left[3] 
right = final_coloc_snp_genes_loops[[17,18,19,6]].T.reset_index(drop=True).T
right[3] = 'R-' + right[3] 
anchors = pd.concat([left, right], ignore_index=True, axis=0)


# In[58]:


# In[54]:
coloc_anchors_fn = os.path.join(outdir, 'coloc.anchors_only.bed')
bed_WashU_bedgz(coloc_anchors_fn, anchors)


# # make the hub json file

#gwas, cline = coloc_fn.split('/')[5:7]

print("# make the hub json file")

gene_refbed_json = {'type': 'refbed',
                    'filename': os.path.basename(sg_genes_fn) + '.gz',
                    'name': 'Gencode V19',
                    'showOnHubLoad': True
                   }

orig_loops_json = {'type': 'longrange',
                   'filename': os.path.basename(loop_gz_link),
                   'name': 'Original Loops',
                   'options': {'displayMode': 'arc', 'color':'red'},
                   'showOnHubLoad': True
                }

# fivekb_snp_gene_pairs_json = {'type': 'longrange',
#                    'filename': os.path.basename(fivekb_snp_gene_pairs_fn) + '.gz',
#                    'name': '5kb SNP-Gene Pairs',
#                    'options': {'displayMode': 'arc', 'color':'purple', 'height': 200},
#                    'showOnHubLoad': False
#                 }

# fivekb_snp_gene_loops_json = {'type': 'longrange',
#                    'filename': os.path.basename(fivekb_snp_gene_loops_fn) + '.gz',
#                    'name': '5kb SNP-Gene Loops',
#                    'options': {'displayMode': 'arc', 'color':'red', 'height': 200},
#                    'showOnHubLoad': False
#                 }


eqtl_snp_gene_pairs_json = {'type': 'longrange',
                 'filename': os.path.basename(eqtl_snp_gene_pairs_fn) + '.gz',
                 'name': 'eQTL SNP-Gene Pairs',
                 'options': {'displayMode': 'arc', 'color':'purple', 'height': 200},
                 'showOnHubLoad': True
                }


eqtl_snps_only_json = {'type': 'bed',
                 'filename': os.path.basename(eqtl_snps_only_fn) + '.gz',
                 'name': 'eQTL SNPs only',
                 'options': {'color':'purple'},
                 'showOnHubLoad': True

                }


eqtl_genes_only_json = {'type': 'bed',
                 'filename': os.path.basename(eqtl_genes_only_fn) + '.gz',
                 'name': 'eQTL genes only',
                 'options': {'color':'purple'},
                 'showOnHubLoad': True
                }


coloc_snp_gene_pairs_json = {'type': 'longrange',
                 'filename': os.path.basename(coloc_snp_gene_pairs_fn) + '.gz',
                 'name': 'coloc SNP-Gene Pairs',
                 'options': {'displayMode': 'arc', 'color':'purple', 'height': 200},
                 'showOnHubLoad': True
                }

coloc_snp_gene_loops_json = {'type': 'longrange',
                 'filename': os.path.basename(coloc_snp_gene_loops_fn) + '.gz',
                 'name': 'coloc SNP-Gene Loops',
                 'options': {'displayMode': 'arc', 'color':'red', 'height': 200},
                 'showOnHubLoad': True
                }

coloc_snps_only_json = {'type': 'bed',
                 'filename': os.path.basename(coloc_snps_only_fn) + '.gz',
                 'name': 'coloc SNPs only',
                 'options': {'color':'purple'},
                 'showOnHubLoad': False
                }


coloc_genes_only_json = {'type': 'bed',
                 'filename': os.path.basename(coloc_genes_only_fn) + '.gz',
                 'name': 'coloc genes only',
                 'options': {'color':'purple'},
                 'showOnHubLoad': False
                }


coloc_anchors_only_json = {'type': 'bed',
                 'filename': os.path.basename(coloc_anchors_fn) + '.gz',
                 'name': 'coloc loop anchors only',
                 'options': {'color':'red'},
                 'showOnHubLoad': False
                }


#hub_json = [orig_loops_json, sg_pairs_json, sg_loops_json, sg_snps_json, sg_genes_json]
hub_json = [gene_refbed_json, 
            #fivekb_snp_gene_pairs_json,
            #fivekb_snp_gene_loops_json,
            eqtl_snps_only_json,
            eqtl_genes_only_json,
            eqtl_snp_gene_pairs_json,
            coloc_snps_only_json,
            coloc_genes_only_json,
            coloc_snp_gene_pairs_json, 
            coloc_anchors_only_json,
            coloc_snp_gene_loops_json,
            orig_loops_json, 
           ]

hub_json_fn = os.path.join(outdir, 'hub.config.json')
with open(hub_json_fn, 'w') as f:
    f.write(json.dumps(hub_json, indent=4))